# 2 - Pre processing des données

In [22]:
# Import des bibliothèques

import pandas as pd
import numpy as np
import dateparser

import requests
from bs4 import BeautifulSoup

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
import re

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from pyLDAvis import prepare
from sklearn.decomposition import LatentDirichletAllocation



## 2-1 Nettoyage des données 

In [23]:
# import de l export .csv

df=pd.read_csv("legifrance_ert.csv", sep=";")
df.head()

,N° Titre,Titre,N° Chapitre,Chapitre,N° Section,Section,Article,Mise en application,Contenu,Nombre phrases,Numéro Décret,Date decret,Annee decret
0,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),1,Champ d'application et définitions (Articles R...,Article R4211-1,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Les dispositions du présent titre déterminent,...",1,2008-244,2008-03-07,2008
1,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),1,Champ d'application et définitions (Articles R...,Article R4211-2,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Pour l'application du présent titre, on entend...",2,2008-244,2008-03-07,2008
2,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),2,Dossier de maintenance (Articles R4211-3 à R42...,Article R4211-3,Modifié par Décret n°2011-1461\n du 7 novembre...,Le maître d'ouvrage élabore et transmet aux ut...,2,2011-1461,2011-11-07,2011
3,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),2,Dossier de maintenance (Articles R4211-3 à R42...,Article R4211-4,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...,1,2008-244,2008-03-07,2008
4,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),2,Dossier de maintenance (Articles R4211-3 à R42...,Article R4211-5,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...,1,2008-244,2008-03-07,2008


In [24]:
# nettoyage caractères spéciaux et stop words en francais

nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower()  
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'[^\w\s]', '', text)  
    text = " ".join([word for word in text.split() if word not in stop_words])  
    return text

df['text_cleaned'] = df['Contenu'].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrateur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df.head()

,N° Titre,Titre,N° Chapitre,Chapitre,N° Section,Section,Article,Mise en application,Contenu,Nombre phrases,Numéro Décret,Date decret,Annee decret,text_cleaned
0,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),1,Champ d'application et définitions (Articles R...,Article R4211-1,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Les dispositions du présent titre déterminent,...",1,2008-244,2008-03-07,2008,dispositions présent titre déterminent applica...
1,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),1,Champ d'application et définitions (Articles R...,Article R4211-2,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Pour l'application du présent titre, on entend...",2,2008-244,2008-03-07,2008,lapplication présent titre entend lieux travai...
2,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),2,Dossier de maintenance (Articles R4211-3 à R42...,Article R4211-3,Modifié par Décret n°2011-1461\n du 7 novembre...,Le maître d'ouvrage élabore et transmet aux ut...,2,2011-1461,2011-11-07,2011,maître douvrage élabore transmet utilisateurs ...
3,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),2,Dossier de maintenance (Articles R4211-3 à R42...,Article R4211-4,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...,1,2008-244,2008-03-07,2008,dossier maintenance lieux travail indique lors...
4,1.0,Obligations du maître d'ouvrage pour la concep...,1.0,Principes généraux (Articles R4211-1 à R4211-5),2,Dossier de maintenance (Articles R4211-3 à R42...,Article R4211-5,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...,1,2008-244,2008-03-07,2008,dossier maintenance lieux travail tenu disposi...


In [15]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# essai 1 

In [52]:
# tokenisation - lemmatisation avec spacy 

nlp = spacy.load("fr_core_news_md")
df['Contenu_processed'] = df['Contenu'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))


In [37]:
pip install gensim

  Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl.metadata (8.2 kB)
Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl (24.0 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
# vectorisation - embeding sur nos données traitées avec méthode TD-IDF

tfidf_vec = TfidfVectorizer()
tfidf_contenu = tfidf_vec.fit_transform(df['Contenu'])


In [ ]:

tfidf_contenu_bis = pd.DataFrame(tfidf_contenu.toarray(), columns=tfidf_vec.get_feature_names_out())
print(tfidf_contenu_bis.head(3))

In [57]:
# vectorisation - embeding sur nos données traitées avec méthode gensim

w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=100,
                     alpha=0.03, 
                     negative=10,
                     seed=1)

In [58]:
sentences = [str(line).split() for line in df['Contenu']]
w2v_model.build_vocab(sentences, progress_per=10000)
words = list(w2v_model.wv.index_to_key)

print('Vocabulary size: %d' % len(words))
print(words)

Vocabulary size: 64
['travail', 'locaux', 'r', 'travailleurs', 'dispositions', 'installations', 'personnes', 'scurit', 'larticle', '1', '2', 'tre', 'lemployeur', 'cas', 'dune', 'dun', 'risques', 'lectriques', 'lorsque', 'articles', 'manire', 'lieux', 'local', 'portes', 'ainsi', 'conditions', 'mesures', 'btiments', 'emplacements', 'plus', 'notamment', 'peuvent', 'rgles', 'protection', 'lagriculture', 'dgagements', 'contre', '3', 'arrt', 'disposition', 'chapitre', 'relatives', 'leurs', 'tablissements', 'ministres', 'prvues', 'conus', 'moins', 'construction', 'chaque', 'chargs', 'mtre', 'sant', 'nature', 'prsent', 'aprs', 'caractristiques', 'compte', 'dont', 'lair', 'nombre', 'lextrieur', 'circulation', 'fixes']


In [59]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30)

(22064, 264780)

In [60]:
x = w2v_model.wv['risques']
print('Shape of one vector: ', x.shape, '\n')
print(x [:15])

Shape of one vector:  (100,) 

[ 0.09311874  0.12823166 -0.11902932  0.06800435  0.03670401 -0.22446293
  0.13440385  0.37896368 -0.191868   -0.21064891  0.01552475 -0.20254067
 -0.03435975  0.19287196  0.14193581]


In [61]:
model.save('w2v_model.bin')

In [ ]:
# essai 2

In [25]:
# vectorisation - embeding sur nos données traitées avec méthode CountVectorizer

data = df['text_cleaned'].tolist()
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=french_stop_words)
X = vectorizer.fit_transform(data)

In [26]:
# application du topic modeling, on instancie un modèle

lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(X)

LatentDirichletAllocation(n_components=5, random_state=0)

In [27]:
# visualisation 
lda_data = pyLDAvis.prepare(
    topic_term_dists=lda.components_,
    doc_topic_dists=lda.transform(X),
    doc_lengths=[len(text.split()) for text in data],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=X.sum(axis=0).A1
)

pyLDAvis.display(lda_data)
